In [9]:
import requests     
import pandas as pd 
import io

In [10]:
# Building blocks for the URL
entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
resource = 'data'           # The resource for data queries is always'data'

flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
key = 'D.CHF.EUR.SP00.A'    # Defining the dimension values, explained below

# Define the parameters
parameters = {
    'startPeriod': '2020-01-01',  # Start date of the time series
    'endPeriod': '2022-12-05'     # End of the time series
}

In [11]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request
response = requests.get(request_url, params=parameters)

# Check if the response returns succesfully with response code 200
print(response)

# Print the full URL
print(response.url)

<Response [200]>
https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A?startPeriod=2020-01-01&endPeriod=2022-12-05


In [13]:
# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})

# Response succesful? (Response code 200)
print(response)

<Response [200]>


In [14]:
# Read the response as a file into a Pandas DataFrame
df = pd.read_csv(io.StringIO(response.text))

In [15]:
# Check the DataFrame's information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   KEY              753 non-null    object 
 1   FREQ             753 non-null    object 
 2   CURRENCY         753 non-null    object 
 3   CURRENCY_DENOM   753 non-null    object 
 4   EXR_TYPE         753 non-null    object 
 5   EXR_SUFFIX       753 non-null    object 
 6   TIME_PERIOD      753 non-null    object 
 7   OBS_VALUE        753 non-null    float64
 8   OBS_STATUS       753 non-null    object 
 9   OBS_CONF         753 non-null    object 
 10  OBS_PRE_BREAK    0 non-null      float64
 11  OBS_COM          0 non-null      float64
 12  TIME_FORMAT      753 non-null    object 
 13  BREAKS           0 non-null      float64
 14  COLLECTION       753 non-null    object 
 15  COMPILING_ORG    0 non-null      float64
 16  DISS_ORG         0 non-null      float64
 17  DOM_SER_IDS     

In [17]:
# Create a new DataFrame called 'ts'
ts = df.filter(['TIME_PERIOD', 'OBS_VALUE'], axis=1)

# 'TIME_PERIOD' was of type 'object' (as seen in df.info). Convert it to datetime first
ts['TIME_PERIOD'] = pd.to_datetime(ts['TIME_PERIOD'])

# Set 'TIME_PERIOD' to be the index
ts = ts.set_index('TIME_PERIOD')

# Print the first 5 rows to screen
ts.head()

,OBS_VALUE
TIME_PERIOD,
2020-01-02,1.0865
2020-01-03,1.0840
2020-01-06,1.0850
2020-01-07,1.0850
2020-01-08,1.0792
